In [ ]:
import sys

sys.path.append("../src/")
from nnet import CRNN
#from pl_trainer import SEDTask4_2021
import pytorch_lightning as pl
from MT_trainer import CoSMo_benchmark
from utils.encoder import ManyHotEncoder
from utils.utils import batched_decode_preds
import scipy
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import torchaudio
import librosa
import librosa.display
import time
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import h5py
from IPython.display import Audio
from processing.sampler import ConcatDatasetBatchSampler
import torch
from tqdm import tqdm
import speechbrain as sb
import os

In [ ]:
from processing.datasets import ConcatDatasetUrban, HDF5_dataset

In [ ]:
with open("../config/sed.yaml", "r") as f: #try to use sed_hear/ sed_bis previous
    conf = yaml.safe_load(f)

# Load Encoder

In [ ]:
with open(f"../config/taxonomy_SONYC.yaml", "r") as f:
    taxonomy_sn = yaml.safe_load(f)

with open(f"../config/taxonomy_SINGA-PURA.yaml", "r") as f:
    taxonomy_sgp = yaml.safe_load(f)


taxonomy_coarse_sn = taxonomy_sn["coarse"]
taxonomy_fine_sn = taxonomy_sn["fine"]
taxonomy_coarse_sgp = taxonomy_sgp["coarse"]
taxonomy_fine_sgp = taxonomy_sgp["fine"]

encoder_fine_sn = ManyHotEncoder(
    taxonomy_sn,
    use_taxo_fine=True,
    audio_len=conf["data"]["audio_max_len"],
    frame_len=conf["features"]["n_filters"],
    frame_hop=conf["features"]["hop_length"],
    net_pooling=conf["data"]["net_subsample"],
    fs=conf["data"]["fs"],
)
encoder_coarse_sn = ManyHotEncoder(
    taxonomy_sn,
    use_taxo_fine=False,
    audio_len=conf["data"]["audio_max_len"],
    frame_len=conf["features"]["n_filters"],
    frame_hop=conf["features"]["hop_length"],
    net_pooling=conf["data"]["net_subsample"],
    fs=conf["data"]["fs"],
)
encoder_fine_sgp = ManyHotEncoder(
    taxonomy_sgp,
    use_taxo_fine=True,
    audio_len=conf["data"]["audio_max_len"],
    frame_len=conf["features"]["n_filters"],
    frame_hop=conf["features"]["hop_length"],
    net_pooling=conf["data"]["net_subsample"],
    fs=conf["data"]["fs"],
)
encoder_coarse_sgp = ManyHotEncoder(
    taxonomy_sgp,
    use_taxo_fine=False,
    audio_len=conf["data"]["audio_max_len"],
    frame_len=conf["features"]["n_filters"],
    frame_hop=conf["features"]["hop_length"],
    net_pooling=conf["data"]["net_subsample"],
    fs=conf["data"]["fs"],
)
encoder = encoder_coarse_sn

# Load Dataset

## Dataset Prep

In [ ]:
SINGAPURA_train_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_train"], "SINGA-PURA", encoder
)
SONYC_train_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_train"], "SONYC", encoder
)
SINGAPURA_val_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_val"], "SINGA-PURA", encoder
)
unlabelled_SINGAPURA_train_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_train"],
    "unlabelled_SINGA-PURA",
    encoder,
)
SONYC_val_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_val"], "SONYC", encoder
)
SINGAPURA_test_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_test"], "SINGA-PURA", encoder
)
SONYC_test_set = HDF5_dataset(
    conf["data"]["root_path"] + conf["data"]["hdf5_test"], "SONYC", encoder
)


## Dataset Concat